In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [5]:
from datasets import load_dataset

dataset = load_dataset("alexcadillon/SemEval2014Task4", "laptops")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


SemEval2014Task4.py:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/224k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

Generating trial split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/3045 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/800 [00:00<?, ? examples/s]

In [6]:
def expand_aspects(batch):
    texts = []
    labels = []

    for text, aspect_list in zip(batch["text"], batch["aspectTerms"]):
        for aspect in aspect_list:
            polarity = aspect.get("polarity", None)
            term = aspect.get("term", None)
            if polarity in ["positive", "negative", "neutral"] and term:
                texts.append(f"{text} Aspect: {term}")
                labels.append(polarity)

    return {"text": texts, "label": labels}

# Apply the mapping function
expanded_dataset = dataset["train"].map(
    expand_aspects,
    batched=True,
    remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

In [7]:
label2id = {"positive": 0, "negative": 1, "neutral": 2}
id2label = {v: k for k, v in label2id.items()}

expanded_dataset = expanded_dataset.map(lambda x: {"label": label2id[x["label"]]})


Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

In [8]:
print(expanded_dataset[3])
# {'text': 'The food was great. Aspect: food', 'label': 0}


{'text': 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from. Aspect: "sales" team', 'label': 1}


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = expanded_dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",  # You can enable eval split later
    load_best_model_at_end=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-1a30b58d6776>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: panvipraji23 (panvipraji23-andhra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.046000
20,0.993300
30,0.867700
40,0.751300
50,0.847900
60,0.732600


Step,Training Loss
10,1.046000
20,0.993300
30,0.867700
40,0.751300
50,0.847900
60,0.732600
70,0.751800
80,0.598600
90,0.592500
100,0.642500


TrainOutput(global_step=435, training_loss=0.468323372424334, metrics={'train_runtime': 5067.0089, 'train_samples_per_second': 1.369, 'train_steps_per_second': 0.086, 'total_flos': 229801917936384.0, 'train_loss': 0.468323372424334, 'epoch': 3.0})

In [13]:
# Step: Save model after training (to local folder)
model.save_pretrained("absa-laptops-model")
tokenizer.save_pretrained("absa-laptops-model")

('absa-laptops-model/tokenizer_config.json',
 'absa-laptops-model/special_tokens_map.json',
 'absa-laptops-model/vocab.txt',
 'absa-laptops-model/added_tokens.json',
 'absa-laptops-model/tokenizer.json')

In [17]:
from google.colab import drive
drive.mount('/content/drive')

# Save inside Google Drive
model.save_pretrained("/content/drive/MyDrive/absa-laptops-model")
tokenizer.save_pretrained("/content/drive/MyDrive/absa-laptops-model")


Mounted at /content/drive


('/content/drive/MyDrive/absa-laptops-model/tokenizer_config.json',
 '/content/drive/MyDrive/absa-laptops-model/special_tokens_map.json',
 '/content/drive/MyDrive/absa-laptops-model/vocab.txt',
 '/content/drive/MyDrive/absa-laptops-model/added_tokens.json',
 '/content/drive/MyDrive/absa-laptops-model/tokenizer.json')

In [14]:
def predict_sentiment(text, aspect):
    input_text = f"{text} Aspect: {aspect}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax(dim=1).item()
    return id2label[predicted_class]

# Test example
print(predict_sentiment("The battery drains quickly", "battery"))


negative


In [15]:
pip install gradio

In [16]:
import gradio as gr

def gradio_interface(review, aspect):
    return predict_sentiment(review, aspect)

gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Laptop Review", placeholder="The screen is bright but the battery is weak."),
        gr.Textbox(label="Aspect Term", placeholder="battery"),
    ],
    outputs=gr.Textbox(label="Predicted Sentiment"),
    title="Aspect-Based Sentiment Analysis (Laptops)",
    description="Enter a laptop review and an aspect (like 'battery', 'keyboard', etc.) to predict sentiment for that aspect."
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://00f169f919aa573075.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
